In [16]:
%config Completer.use_jedi=False
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
sys.path.append("../")

In [3]:
import base64
import requests
import torch
import os
import numpy as np
import argparse
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable
import torchvision.utils as vutils
from network.Transformer import Transformer

from tqdm import tqdm_notebook
from test_from_code import transform

import onnx
import onnxruntime

In [4]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
styles = ["Hosoda", "Hayao", "Shinkai", "Paprika"]

models = {}

for style in tqdm_notebook(styles):
    model = Transformer()
    model.load_state_dict(torch.load(os.path.join("./../pretrained_models/", style + '_net_G_float.pth')))
    model.eval()
    models[style] = model

/Users/ahmedbesbes/anaconda3/envs/pt/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [30]:
model = models["Hosoda"]
#model = model.float()
model.eval();

In [31]:
x = torch.randn(1, 3, 224, 224, requires_grad=True)

In [42]:
%%time
output = model(x)

CPU times: user 3.13 s, sys: 398 ms, total: 3.53 s
Wall time: 3.8 s


In [84]:
torch.onnx.export(model, 
                  x, 
                  "Hosoda.onnx", 
                  export_params=True, 
                  opset_version=10, 
                  do_constant_folding=True, 
                  input_names=["input"], 
                  output_names=["output"], 
                  dynamic_axes={"input": {0: "batch_size", 2: "width", 3: "height"}})

In [85]:
onnx_model = onnx.load("Hosoda.onnx")

In [86]:
onnx.checker.check_model(onnx_model)

In [87]:
ort_session = onnxruntime.InferenceSession("Hosoda.onnx")

In [47]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [67]:
x.shape


torch.Size([1, 3, 224, 224])

In [101]:
array = np.random.random((1, 3, 312, 450))
array = np.array(array, dtype=np.float32)
ort_inputs = {ort_session.get_inputs()[0].name: array}

In [102]:
%%time
ort_outs = ort_session.run(None, ort_inputs)

CPU times: user 17.2 s, sys: 187 ms, total: 17.4 s
Wall time: 5.54 s
